# Text Classification with Deep Learning using BERT

## Exploratory Data Analysis and Preprocessing

In [0]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [0]:
train_df = pd.read_csv('/content/IDAT_training_text.csv')
train_df.set_index('id', inplace=True)

In [0]:
train_df.head()

,text,label,type_,tweet_id
id,,,,
0,ايمان عز الدين:الجراد طلع علي المقطم وبعدين بي...,1,training,'308488170838831104'
1,@AymanNour الى المدعو أيمن نور الحرامى من معك ...,0,training,'955724773216129024'
2,#بوتين ٦٥ سنه و بيغطس في بحيره متجمده و انا خا...,0,training,'954792171521048576'
3,#قال أيه أنهاردة 20 مليون واحد في الشوارع عشان...,1,training,'363321598431862784'
4,@EmmanuelMacron وفي كل مره يرفض إيمانويل دعوة ...,0,training,'939204686632103936'


In [0]:
train_df.label.value_counts()

1    2091
0    1933
Name: label, dtype: int64

In [0]:
test_df = pd.read_csv('/content/IDAT_test_text.csv')
test_df.set_index('id', inplace=True)

In [0]:
test_df.head()

,text,label,type_,tweet_id
id,,,,
0,#يناير_حلم_ومكملينه فاستبشروا خيرا واستكملوا ث...,0,test,'955879051872350209'
1,#الشيخه_موزا_مصدر_فخرنا موزه ويسبّــق اسمــها ...,0,test,'953563403368452096'
2,معلش سؤال بس. هو حد علق من جبهة الانقاذ عن احد...,1,test,'322085724235132928'
3,ههههههههههههههههههههه. اه يادماغي هو الاخوان ا...,1,test,'367053834235183104'
4,ايمن نور فى حوار #مرسي اللى كان مذاع والعالم ك...,1,test,'341890733990633473'


In [0]:
test_df.label.value_counts()

1    523
0    483
Name: label, dtype: int64

## Loading Tokenizer and Encoding our Data

In [0]:
!pip install transformers
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset

In [0]:
tokenizer = AutoTokenizer.from_pretrained('asafaya/bert-base-arabic', 
                                          do_lower_case=True)

In [0]:
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    test_df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test_df.label.values)

In [0]:
print(input_ids_train)
print(attention_masks_train)

tensor([[    2,  7441,  3600,  ...,     0,     0,     0],
        [    2,    36, 25600,  ...,     0,     0,     0],
        [    2,     7, 10359,  ...,     0,     0,     0],
        ...,
        [    2, 26894,  1774,  ...,     0,     0,     0],
        [    2,  3551,  1788,  ...,     0,     0,     0],
        [    2,     7,  8371,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [0]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [0]:
len(dataset_train)

4024

In [0]:
len(dataset_val)

1006

## Setting up BERT Pretrained Model

In [0]:
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained("asafaya/bert-base-arabic",
                                                      num_labels=len(set(train_df.label.values)),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

## Creating Data Loaders

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [0]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Setting Up Optimiser and Scheduler

In [0]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [0]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [0]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [0]:
import numpy as np
from sklearn.metrics import classification_report

In [0]:
label_dict = {0: 0 , 1: 1}
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [0]:
def accuracy(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return classification_report(labels_flat, preds_flat)

## Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [0]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [0]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [0]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score: {val_f1}')


Epoch 1
Training loss: 0.48834628363450366
Validation loss: 0.44042668654583395
F1 Score (Weighted): 0.802747791952895



Epoch 2
Training loss: 0.3659812287678794
Validation loss: 0.419376477599144
F1 Score (Weighted): 0.8328611898016995



Epoch 3
Training loss: 0.30868087023023577
Validation loss: 0.4222607589326799
F1 Score (Weighted): 0.8367346938775511



In [0]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [0]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [0]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 379/483

Class: 1
Accuracy: 451/523



In [0]:
print(accuracy(predictions, true_vals))

              precision    recall  f1-score   support

           0       0.84      0.78      0.81       483
           1       0.81      0.86      0.84       523

    accuracy                           0.83      1006
   macro avg       0.83      0.82      0.82      1006
weighted avg       0.83      0.83      0.82      1006

